In [2]:
!pip install unidecode
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime
from sys import path
import pandas as pd
from openpyxl.descriptors import serialisable
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill, Font, Border, Alignment
from openpyxl.styles import colors
from openpyxl.cell import Cell
from openpyxl.utils import get_column_letter
from openpyxl.styles import Side, Border
from unidecode import unidecode
import PIL
from PIL import Image, ImageDraw, ImageFont
from google.colab import files
import glob
import textwrap
from datetime import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import os.path
from os import path
from google.colab import files
import json
import sys
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)


def envia_email(receiver_address, subject, file_path, data):
  mail_content = f"Olá, {data['aluno']}. {data['mensagem']}"
  sender_address = 'projetostem@uea.edu.br'
  sender_pass = 'jexhalnqfqlbxrui'
  
  #Setup the MIME
  message = MIMEMultipart()
  message['From'] = sender_address
  message['To'] = receiver_address
  message['Subject'] = subject


  #The subject line
  #The body and the attachments for the mail
  message.attach(MIMEText(mail_content, 'plain'))

  '''attach_file_name = '/content/gundam unicorn - head.pdf'''
  attach_file_name = file_path
  if file_path != '': 
    attach_file = open(attach_file_name, 'rb') # Open the file as binary mode
    payload = MIMEBase('application', 'pdf', Name=f"Certificado_{data['aluno']}.pdf")
    payload.set_payload((attach_file).read())
    encoders.encode_base64(payload) #encode the attachment
    #add payload header with filename
    payload.add_header('Content-Decomposition', 'attachment', filename=attach_file_name)
    message.attach(payload)
  else: 
    pass

  '''payload = MIMEBase('application', 'octate-stream')'''


  html = """<!DOCTYPE html>
  <html>
  <title>Web Page Design</title>
  <head>
      
  <style type="text/css">
      h5 {font-family: Arial;font-size:10px}

  </style>

  </head>

  <body>
      <div style='display: table-cell; vertical-align: middle;'></div>
          <a href="stem.uea.edu.br"><img style="width: 150px; height: 150px; display: block; float:left;" src="https://i.ibb.co/V95xQdq/ACAD-STEM-transparente-1.png" alt="ACAD-STEM-transparente-1" border="0" title="STEM"></a>
          <h5 style="color:green;">Projeto Academia STEM</h5>
          <h5>Universidade do Estado do Amazonas</h5>
          <h5>Escola Superior de Tecnologia</h5>
          <a href="stem.uea.edu.br" style="font-family: Arial; margin-bottom: 0px; font-size:10px">stem.uea.edu.br</a>
      </div>
  </body>
  </html>"""

  part = MIMEText(html, "html")
  message.attach(part)
  #Create SMTP session for sending the mail
  session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
  session.starttls() #enable security
  session.login(sender_address, sender_pass) #login with mail_id and password
  text = message.as_string()
  session.sendmail(sender_address, receiver_address, text)
  session.quit()
  print('Mail Sent')

def carrega_img(path_template):
  return Image.open(path_template)

def bbox(img):
  im = img.copy()
  bbox = ImageDraw.Draw(im) 
  w, h = im.size # tamanho
  ## codigo
  cod = [((w)/7.3, 105.5), (w/5.2, 127)] #((onde comeca), (onde termina))
  bbox.rectangle(cod, outline='red')

  ## aluno
  aluno = [((w)/8.1, 280), (w/1.14, 380)] #((onde comeca), (onde termina))
  bbox.rectangle(aluno, outline='red')
  
  ## ano
  p = 580
  ano = [((w/92)+p, 445), ((w/14)+p, 462)] #((onde comeca), (onde termina))
  bbox.rectangle(ano, outline='red')
  
  ## mes
  p = 470
  mes = [((w/35)+p, 445), ((w/12)+p, 462)] #((onde comeca), (onde termina))
  bbox.rectangle(mes, outline='blue')

  ## dia
  dia = [((w)/2.4, 445), (w/2.245, 462)] #((onde comeca), (onde termina))
  bbox.rectangle(dia, outline='blue')

  return im


def GeraCod_un(n, un = 0):
  # un =len do anterior
  num = un
  n['codigo'] = []
  if 'presenca' in n.keys():
    for p in n['presenca']:
      num += 1
      n['codigo'].append(f"PA{int(str(int(num/1000))[-1])}{int(str(int(num/100))[-1])}{int(str(int(num/10))[-1])}{int(str(int(num/1))[-1])}")
    n['ganha_certificado'] = []
    for p in n['presenca']:
      if p >=70:
        n['ganha_certificado'].append(True)
      else:
        n['ganha_certificado'].append(False)

    return n
  else:
    for p in n['nome']:
      num += 1
      n['codigo'].append(f"PA{int(str(int(num/1000))[-1])}{int(str(int(num/100))[-1])}{int(str(int(num/10))[-1])}{int(str(int(num/1))[-1])}")
    n['ganha_certificado'] = []
    for p in n['nome']:
      n['ganha_certificado'].append(True)
    return n

def gerar_certificado(im2, aluno, cod, text, path_fonte = '/content/drive/MyDrive/Certificados-laboratorios-moveis/Cursos Carreta/fontes/Poppins-Black.ttf',
                      fonte2 = '/content/drive/MyDrive/Certificados-laboratorios-moveis/Cursos Carreta/fontes/Poppins-Regular.ttf'):
  im = im2.copy()

  draw = ImageDraw.Draw(im) 
  font = ImageFont.truetype(path_fonte, 26) 
 
  ## Escrevendo o nome
  nome = aluno.upper()
  para = textwrap.wrap(nome, width=30)
  w, h = draw.textsize(nome)
  current_h, pad = 280, 10

  if len(para) > 1:
    current_h, pad = 280, 10
  else: 
    current_h, pad = 310, 10
  for line in para:
    
    w, h = draw.textsize(line, font=font)
    draw.text(((im.size[0] - w) / 2, current_h), line,(0, 0, 0), font=font)
    current_h += h + pad


  ## Escrevendo o codigo
  font = ImageFont.truetype(path_fonte, 18) 
  codigo = cod
  w, h = draw.textsize(codigo)
  pos_cod = ((w*36)/8.9, 104.5) 
  draw.text(pos_cod, codigo, (0, 0, 0), font = font)


  # Escrevendo o texto
  font = ImageFont.truetype(fonte2, 16) 
  texto = text
  para = textwrap.wrap(texto, width=69)
  w, h = draw.textsize(texto)
  current_h, pad = 400, 10
  #current_h = altura (é medido de cima para baixo, entao quanto maior, mais em baixo está)
  #pad = distancia entre linhas

  for line in para:
    w, h = draw.textsize(line, font=font)
    draw.text(((im.size[0] - w) / 2, current_h), line,(0, 0, 0), font=font)
    current_h += h + pad

  return im

def salva_certificados(aprovados, template, un, texto, nome_curso):
  aprovados['Certificados'] = []

  for ind in range(len(aprovados['nome'])):

    aprovados['Certificados'].append(gerar_certificado(template, 
                                                       aprovados['nome'][ind], 
                                                       aprovados['codigo'][ind],
                                                       texto))

  path = f"/content/drive/MyDrive/Certificados-laboratorios-moveis/Cursos Carreta/certificados/{nome_curso}"
  
  try:
    os.mkdir(path)
  except:
    pass
    

  i= un
  aprovados["Path_certificados"] = []
  c = 0 
  for certificado in aprovados['Certificados']:
    i+=1
    certificado = certificado.convert("RGB")
    certificado.save(path+f"/{aprovados['nome'][c]}.pdf", format='pdf')
    aprovados["Path_certificados"].append(path+f"/{aprovados['nome'][c]}.pdf")
    c+=1
  return aprovados

def envia_aprovados(subject, aprovados, msg):
  # Mensagem = {"mensagem": 'mensagem a ser enviada', 'aluno': 'nome do fulano'}
  for i in range(len(aprovados['nome'])):
    if aprovados['ganha_certificado'][i]:
      try:
        envia_email(aprovados['email'][i], 
                    subject,aprovados['Path_certificados'][i],
                    {"mensagem":msg, "aluno": aprovados["nome"][i]})
      except:
        print(f"Erro: email inválido {aprovados['email'][i]}")
    else:
      pass

  return "sucess"

def tipocurso(ch):
  if ch <=4:
    return 'minicurso'
  else:
    return 'curso'

def coleta_data(data0, data1):

  mes = ("Janeiro","Fevereiro","Março","Abril","Maio","junho","Julho","Agosto",
         "Setembro","Outubro","Novembro","Dezembro")
  
  # Data0 é quando o curso comecou, data1 quando terminou
  d0 = pd.Timestamp(data0[0]).day
  m0 = pd.Timestamp(data0[0]).month
  a0 = pd.Timestamp(data0[0]).year

  d1 = pd.Timestamp(data1[0]).day
  m1 = pd.Timestamp(data1[0]).month
  a1 = pd.Timestamp(data1[0]).year
  t = 'n'
  if d0 == d1 and m0 == m1 and a0 == a1:
    t = '1'
    texto = f"em {d0} de {mes[m0-1]} de {a0}"
    
  elif (not (d0 == d1)) and (m0 == m1) and (a0 == a1):
    texto = f"de {d0} a {d1} de {mes[m1-1]} de {a1}"

  elif (not (d0 == d1)) and (not (m0 == m1)) and (a0 == a1):
    texto = f"de {d0} de {mes[m0-1]} a {d1} de {mes[m1-1]} de {a1}"
  
  elif (not (d0 == d1)) and (not (m0 == m1)) and (not(a0 == a1)):
    texto = f"de {d0} de {mes[m0-1]} de {a0} a {d1} de {mes[m1-1]} de {a1}"
  

  return texto,  t

def coleta_dia(d):
    dia = {"Monday": "segunda", "Tuesday":"terca", "wednesday":"quarta", 
        "Thursday":"quinta", "Friday": "sexta", "Saturday": "Sabado",
        "Sunday": "domingo"}

    return dia[pd.Timestamp(d0[0]).day_name()]

def gera_Data(string):
  d = datetime.strptime(string, '%d/%m/%Y %H:%M:%S')
  return f"{d.day}/{d.month}/{d.year}" 
  
def checa_intervalo(preenchimento, comeco, fim, limite_inf, limite_sup):
  preenchimento = datetime.strptime(preenchimento, '%d/%m/%Y %H:%M:%S')
  if comeco.month == fim.month and comeco.year == fim.year:
    # se foi preenchido um dia depois ou um dia antes

    if preenchimento.day > fim.day or preenchimento.day < comeco.day:
      return False

  if comeco.month == fim.month and comeco.year != fim.year:
    # se foi preenchido um dia depois ou um dia antes
    if preenchimento.day > fim.day or preenchimento.day < comeco.day:
      return False  

  if comeco.month != fim.month and comeco.year == fim.year:
    # se foi preenchido um dia depois ou um dia antes
    if preenchimento.month > fim.month or preenchimento.month < comeco.month: 
      if preenchimento.day > fim.day or preenchimento.day < comeco.day:
        return False
  if preenchimento.hour == limite_sup:
    if preenchimento.minute >= 0:
      if preenchimento.second > 0:
        return False
  if preenchimento.hour < limite_inf or preenchimento.hour > limite_sup:
    return False
  return True

def coleta_presenca(forms, turno, d0, d1, d = 1):
  lidos = [] # Salvar nome e data da leitura, para nao
  if turno == 'TARDE':
    limite_inf = 13
    limite_sup = 17

  else:
    limite_inf = 7
    limite_sup = 12
    
  f = forms.copy()
  f[f.columns[0]] = pd.to_datetime(f[f.columns[0]]).dt.date
  d_ini = d0[0].date()
  d_fim = d1[0].date()
  _, t = coleta_data(d0, d1)
  coluna_data = f.columns[0]
  coluna_email = f.columns[1]
  coluna_nome = f.columns[2]
  coluna_serie = f.columns[3]

  #Para saber o nome das colunas que estão entrando
  #print(f.columns)
  
  if t == '1':
    presentes = {"nome": [], "email": [], "serie": []}
    # mini curso de 1 dia
    for aluno in range(len(forms)):
      preenchimento = forms[coluna_data][aluno]
      if checa_intervalo(preenchimento, d0[0], d1[0], limite_inf, limite_sup):
        presentes["email"].append(f[coluna_email][aluno])
        presentes["nome"].append(f[coluna_nome][aluno])
        presentes["serie"].append(f[coluna_serie][aluno])
    return presentes
  
  elif t == 'n':
    presentes = {"nome":[], "email":[], "presenca":[], "serie":[]}
    # curso de mais de 1 dia
    dias = []
    for aluno in range(len(forms)):
      preenchimento = forms[coluna_data][aluno]
      data= gera_Data(preenchimento)
      reg = (f[coluna_email][aluno], data) #(nome, dd/mm/aaaa)
      if reg not in dias:
        dias.append(reg)
        if checa_intervalo(preenchimento, d0[0], d1[0], limite_inf, limite_sup):
          if f[coluna_email][aluno] in presentes["email"]:
            presentes["presenca"][presentes["email"].index(f[coluna_email][aluno])]+=1
          else:
            presentes["nome"].append(f[coluna_nome][aluno])
            presentes["presenca"].append(1)
            presentes["email"].append(f[coluna_email][aluno])
            presentes["serie"].append(f[coluna_serie][aluno])

    # deixando em %
    #presenca deve ser maior que 70, mas isso vamos ver depois
    for presenca in range(len(presentes['presenca'])):
      presentes['presenca'][presenca] = (presentes['presenca'][presenca]/d)*100
    return presentes

def cria_cabecalho(ws, info):

  ws.column_dimensions['A'].width = 25
  ws.column_dimensions['B'].width = 25
  ws.column_dimensions['C'].width = 25
  ws.column_dimensions['D'].width = 25

  ws.merge_cells('B1:D1')
  ws.merge_cells('B2:D2')
  ws.merge_cells('B4:D4')
  ws.merge_cells('B5:D5')
  
  ws.column_dimensions.group(start='E', end='ZZZ', hidden=True)

  # Celulas de iformações

  aluno = ws['A6']
  presenca = ws['C6']
  serie = ws['D6']
  curso = ws['A1']
  nome_curso = ws['B1']
  escola = ws['A2']
  nome_escola = ws['B2']
  inicio = ws['A3']
  data_inicio = ws['B3']
  fim = ws['C3']
  data_fim = ws['D3']
  aulas = ws['A4']
  n_aulas = ws['B4']
  carreta = ws['A5']
  carreta_nome = ws['B5']


  aluno.value = "aluno"
  ws.merge_cells('A6:B6')
  celula(aluno)

  presenca.value = "presenca"
  celula(presenca)
  serie.value = "serie_escolar"
  celula(serie)

  curso.value = "CURSO"
  celula(curso)

  nome_curso.value = info['curso']

  escola.value = "ESCOLA"
  celula(escola)

  nome_escola.value = info['escola']
  

  inicio.value = "INICIO" 
  celula(inicio)

  data_inicio.value = info['inicio']

  fim.value = "FIM" 
  celula(fim)

  data_fim.value = info['fim']

  aulas.value = "AULAS"
  celula(aulas)

  n_aulas.value = info['aulas']

  carreta.value = "CARRETA"
  celula(carreta)

  carreta_nome.value = info['carreta']

def celula(cell):

  font = Font(name='Calibri',
                 size=11,
                 bold=False,
                 italic=False,
                 vertAlign=None,
                 underline='none',
                 strike=False,
                 color='FF000000')
  
  alignment=Alignment(horizontal='center',
                     vertical='center',
                     text_rotation=0,
                     wrap_text=False,
                     shrink_to_fit=True,
                     indent=0)
  
  fill = PatternFill(start_color='ADD8E6',
                    end_color='ADD8E6',
                    fill_type='solid')
  
  
  cell.font = font
  cell.alignment = alignment
  cell.fill = fill

def celula_valores(cell):

  font = Font(name='Calibri',
                 size=11,
                 bold=False,
                 italic=False,
                 vertAlign=None,
                 underline='none',
                 strike=False,
                 color='FF000000')
  
  alignment=Alignment(horizontal='center',
                     vertical='center',
                     text_rotation=0,
                     wrap_text=False,
                     shrink_to_fit=True,
                     indent=0)
  
  
  cell.font = font
  cell.alignment = alignment

def preenche_alunos(ws, presenca_alunos):
  if 'presenca' in presenca_alunos.keys():
    #Curso completo

    i = 7
    
    for aluno in range(len(presenca_alunos['nome'])):
      nome = ws['A'+str(i)]
      ws.merge_cells(f'A{i}:B{i}')

      presenca = ws['C'+str(i)]
      serie_escolar = ws['D'+str(i)]

      nome.value = unidecode(str(presenca_alunos['nome'][aluno]).upper().rstrip())
      presenca.value = str(round(presenca_alunos['presenca'][aluno],2))+"%"
      serie_escolar.value = presenca_alunos['serie'][aluno]
      
      celula_valores(nome)
      celula_valores(presenca)
      celula_valores(serie_escolar)
      

      i+=1
    pass

  else:
    #minicurso
    i = 7
    for aluno in range(len(presenca_alunos['nome'])):
      nome = ws['A'+str(i)]
      ws.merge_cells(f'A{i}:B{i}')

      presenca = ws['C'+str(i)]
      serie_escolar = ws['D'+str(i)]

      celula_valores(serie_escolar)
      nome.value = unidecode(str(presenca_alunos['nome'][aluno]).upper().rstrip())
      presenca.value = "100%"
      serie_escolar.value = presenca_alunos['serie'][aluno]
      
      celula_valores(nome)
      celula_valores(presenca)
      celula_valores(serie_escolar)
      celula_valores(serie_escolar)
      

      i+=1

  

def seta_borda(ws, blank=True):
    wb = ws._parent
    side = Side(border_style='thin', color='000000')
    for cell in ws._cells.values():
        cell.border = Border(top=side, bottom=side, left=side, right=side)
    if blank:
        white = Side(border_style='thin', color='FFFFFF')
        wb._borders.append(Border(top=white, bottom=white, left=white, right=white))
        wb._cell_styles[0].borderId = len(wb._borders) - 1

############################################################# leitura dos dados

path_dados_cursos = '/content/drive/MyDrive/Certificados-laboratorios-moveis/Cursos Carreta/dados_curso.xlsx'

dados_curso = pd.read_excel(path_dados_cursos, sheet_name= 'dados curso')
dados_carretas = pd.read_excel(path_dados_cursos, sheet_name= 'Carretas')
dados_cursos_geral = pd.read_excel(path_dados_cursos, sheet_name= 'cronograma', skiprows= [0])

#############################################################          dados
# Dados curso
carreta = dados_curso[dados_curso.columns[1]][0] # carreta
nome_curso = dados_curso[dados_curso.columns[1]][1] # nome 
turno = dados_curso[dados_curso.columns[1]][2] # tarde/manha

# carga horaria em horas

ch = int(dados_cursos_geral[dados_cursos_geral.Nome == nome_curso][dados_cursos_geral.columns[3]]) 

tipo = tipocurso(ch) #minicurso ou curso


########## d0 = quando comecou, d1 = quando terminou
d0 = tuple(dados_cursos_geral[dados_cursos_geral.Nome == nome_curso][dados_cursos_geral.columns[0]]) # Quando comecou
d1 = tuple(dados_cursos_geral[dados_cursos_geral.Nome == nome_curso][dados_cursos_geral.columns[1]]) # Quando terminou

data,_ = coleta_data(d0, d1)

texto_certificado = f'''Participou do {tipo} {nome_curso}, do Pilar Atração do Projeto Academia STEM, com carga horária de {ch} horas {data}'''


path = '/content/drive/MyDrive/Certificados-laboratorios-moveis/Cursos Carreta/'

link = {'Carreta1 ': 'https://docs.google.com/spreadsheets/d/1TeAI-xS0xdF3fsCj3ZffqRw5xRehgDUTbmlMM3BbzjU/edit#gid=1133074893',
 'Carreta2': 'https://docs.google.com/spreadsheets/d/1ICio_xOssJkC5kDD-futyIX-dr6_r60nDMe44N6YWis/edit#gid=818766653'}
                       
wb = gc.open_by_url(link[carreta])
sheet = wb.worksheet('Respostas ao formulário 1')

dado = sheet.get_all_values()
forms = pd.DataFrame(dado, columns = ['Carimbo de data/hora',
                                      'Endereço de e-mail', 
                                      'Nome Completo', 
                                      'Série Escolar',
                                      ]).iloc[
                                                          1: , :
                                                          ].reset_index()
forms.drop(forms.columns[0], axis = 1, inplace= True)
f = forms.copy()




template_certificado =  path+'modelo_certificado/Certificados_ementa.png'

alunos = path+'planilhas_teste/alunos.xlsx'
template = Image.open(template_certificado)


data_path = '/content/drive/MyDrive/Certificados-laboratorios-moveis/Cursos Carreta/fontes/dados.json'
with open(data_path) as json_file:
    i = json.load(json_file)['ultimo_lido']


presenca = coleta_presenca(forms, turno, d0, d1, d = int(dados_cursos_geral[dados_cursos_geral.Nome == dados_curso[dados_curso.columns[1]][1]]['Quantidade de aulas']))
############################# Gerando certificados #############################

#dicionario com codigos
cod = GeraCod_un(presenca, i)
#certificados
aprovados = salva_certificados(cod, template, i, texto_certificado,nome_curso)

################################## planilha ####################################

info = {"curso": 0, "escola": 0, "inicio":0, "fim":0, "aulas":0, "carreta":0}
try:
  # curso
  info["curso"] = str(nome_curso)
  #aulas
  info["aulas"] = int(
                          dados_cursos_geral[
                                dados_cursos_geral.Nome == dados_curso[
                                dados_curso.columns[1]][1]]['Quantidade de aulas']
                                )
  #escola 
  info["escola"] = str(dados_carretas[dados_carretas.Carreta == carreta].Escola.values[0])
  #inicio
  info["inicio"] = f'{d0[0].day}/{d0[0].month}/{d0[0].year}'
  #fim 
  info["fim"] = f'{d1[0].day}/{d1[0].month}/{d1[0].year}'
  # Carreta
  info["carreta"] = carreta   
except:
  print("################################################")
  print("################## ATENCAO #####################")
  print("################################################")
  print("informações inválidas, cheque a planilha filtro")
  print("################################################")
  print("################################################")
  sys.exit()

print(f"ESCOLA: {info['escola']}\nTurno: {turno}\nCarreta: {carreta}")
resposta = input("Confirma? (S/N): ")
if resposta.upper() in ('NAO', 'N', 'NÃO', 'NA', 'NOT', 'NEIN'):
  print('certifique-se de atualizar os dados dos arquivos do colab')
  exit()
else:
  wb = Workbook()
  ws = wb.active
  cria_cabecalho(ws, info)

  if presenca['codigo'] != []:
    preenche_alunos(ws, presenca)
    seta_borda(ws)
    print("salvando")
    hj = datetime.today()
    hoje = f"{hj.day}-{hj.month}-{hj.year}"
    wb.save(filename = f'/content/drive/MyDrive/Certificados-laboratorios-moveis/Cursos passados/{nome_curso}_{str(dados_carretas[dados_carretas.Carreta == carreta].Escola.values[0])}_{hoje}.xlsx')
  wb.close()

  #################################    email   ###################################

  assunto = 'PARABÉNS! Certificado do curso Academia STEM.'
  corpo  = f'''
  Parabéns! Você completou o {tipo} {nome_curso}, desenvolvido pela equipe do Pilar Atração nos laboratórios móveis do projeto Academia STEM (AcadSTEM/UEA).

  

  Agradecemos sua participação! Por favor, avalie o nosso curso pelo link abaixo.

  https://forms.gle/Db9Kwv55ScAWhT8X9

  

  Seu CERTIFICADO está em anexo.

  

  Você pode acompanhar o @manostem e @acadstem nas redes sociais e ficar por dentro dos nossos próximos cursos!

  

  Qualquer dúvida, estamos à disposição.
    
  '''
  # Enviar email e salvar último lido

  envia_aprovados(assunto, presenca, corpo)
  if len(aprovados['codigo']) > 0:
    i = int(aprovados['codigo'][-1].replace('PA', ''))
    dados = {'ultimo_lido': i}
    with open(data, 'w') as fp:
        json.dump(dados, fp)
  

# @title Dê o play
!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ESCOLA: IFAM-CMZL
Turno: MANHÃ
Carreta: Carreta2
Confirma? (S/N): s
